In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = Path('/Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


# Split the Data into Training and Testing

In [6]:
# Create our features
X = pd.get_dummies(df[['loan_status', 'total_rec_prncp','last_pymnt_amnt',
                       'total_pymnt','total_pymnt_inv','total_rec_int',
                       'issue_d','int_rate','out_prncp_inv','loan_amnt', 
                       'max_bal_bc','out_prncp']])
# Create our target
y = df.loan_status

In [7]:
X.describe()

,total_rec_prncp,last_pymnt_amnt,total_pymnt,total_pymnt_inv,total_rec_int,int_rate,out_prncp_inv,loan_amnt,max_bal_bc,out_prncp,loan_status_high_risk,loan_status_low_risk,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019
count,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,658.748750,517.202218,970.639473,970.431151,311.855687,0.127718,16015.919519,16677.594562,6297.491710,16018.845833,0.005042,0.994958,0.371696,0.451066,0.177238
std,1126.237377,816.021053,1229.563301,1229.569864,279.786954,0.048130,10004.985940,10277.348590,5702.281757,10002.695495,0.070831,0.070831,0.483261,0.497603,0.381873
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.060000,0.000000,1000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.370000,267.110000,403.760000,403.550000,115.440000,0.088100,8576.500000,9000.000000,2558.000000,8578.800000,0.000000,1.000000,0.000000,0.000000,0.000000
50%,449.890000,408.520000,710.790000,710.790000,225.250000,0.118000,14061.030000,15000.000000,4874.000000,14067.750000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,762.170000,652.580000,1214.600000,1214.590000,415.710000,0.155700,22860.490000,24000.000000,8321.000000,22860.700000,0.000000,1.000000,1.000000,1.000000,0.000000
max,40000.000000,40500.340000,41418.981165,41418.980000,2735.400000,0.308400,40000.000000,40000.000000,207484.000000,40000.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.shape

(51612, 15)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [10]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

random_forest = BalancedRandomForestClassifier(n_estimators = 100)
rf_model = random_forest.fit(X_train, y_train)

In [11]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9996202827433112

In [12]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [   13, 17105]])

In [13]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.87      1.00      1.00      0.93      1.00      1.00        87
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     17118

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205



In [14]:
# List the features sorted in descending order by feature importance
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.3920105505686181, 'loan_status_low_risk'),
 (0.3837967051051034, 'loan_status_high_risk'),
 (0.04493535299440027, 'total_rec_prncp'),
 (0.039213550704160514, 'total_pymnt'),
 (0.0375688525125905, 'total_rec_int'),
 (0.029442479466331703, 'total_pymnt_inv'),
 (0.025838467060533842, 'last_pymnt_amnt'),
 (0.012986233524086599, 'issue_d_Jan-2019'),
 (0.009441665930601915, 'int_rate'),
 (0.007136166052516221, 'issue_d_Mar-2019'),
 (0.005714041914194869, 'out_prncp_inv'),
 (0.004764999589361827, 'max_bal_bc'),
 (0.003314446346919877, 'out_prncp'),
 (0.002166534707555206, 'loan_amnt'),
 (0.0016699535230251914, 'issue_d_Feb-2019')]

### Easy Ensemble AdaBoost Classifier

In [15]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy_model = easy.fit(X_train, y_train)

In [16]:
# Calculated the balanced accuracy score
y_pred = easy_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

1.0

In [17]:
# Display the confusion matrix
y_pred = easy_model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[   87,     0],
       [    0, 17118]])

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       1.00      1.00      1.00      1.00      1.00      1.00        87
   low_risk       1.00      1.00      1.00      1.00      1.00      1.00     17118

avg / total       1.00      1.00      1.00      1.00      1.00      1.00     17205

